{% block title %}
{% endblock %}

## Importing the libraries

In [ ]:
{% block imports %}
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, Conv2D, Conv3D, Dropout, MaxPooling1D, MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D, BatchNormalization
{% endblock %}

## Start stopwatch

In [ ]:
{% block stopwatch %}
from time import process_time
time_start = process_time() 
{% endblock %}

## Import the dataset

In [ ]:
{% block dataset %}
X = pd.read_csv('{{ features_file_path }}')
y = pd.read_csv('{{ labels_file_path }}')
{% endblock %}

## Splitting the dataset into the Training set and Test set

In [ ]:
{% block split %}
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size={{train_split.test_size}},
                        train_size={{train_split.train_size}},
                        random_state={{train_split.random_state}},
                        shuffle={{train_split.random_state}},
                        stratify=X if "{{train_split.stratify}}" == "features" else y if "{{train_split.stratify}}" == "labels" else None)
y_true = y_test
{% endblock %}

## Data preprocessing

In [ ]:
{% block preprocess %}
{% endblock %}

## Build the model

In [ ]:
model = Sequential()
{% for i in range(model.layers|length) %}{% if model.layers[i] == 'flatten' %}model.add(Flatten())
{% elif 'dense' in model.layers[i].keys() %}model.add(Dense({{model.layers[i].dense.units}}{% if model.layers[i].dense.activation%}, activation="{{model.layers[i].dense.activation}}"{% endif %}))
{% elif 'dropout' in model.layers[i].keys() %}model.add(Dropout({{model.layers[i].dropout}}))
{% elif 'convolution' in model.layers[i].keys() %}{% if model.layers[i].convolution.size is integer %}model.add(Conv1D({{model.layers[i].convolution.filters}}, kernel_size={{model.layers[i].convolution.size}}{% if model.layers[i].convolution.activation%}, activation="{{model.layers[i].convolution.activation}}"{% endif %}{% if i == 0 %}{% block input_shape1d %}{% endblock %}{% endif %})){% elif model.layers[i].convolution.size|length == 2 %}model.add(Conv2D({{model.layers[i].convolution.filters}}, kernel_size={{model.layers[i].convolution.size}}{% if model.layers[i].convolution.activation%}, activation="{{model.layers[i].convolution.activation}}"{% endif %}{% if i == 0 %}{% block input_shape2d %}{% endblock %}{% endif %})){% else %}model.add(Conv3D({{model.layers[i].convolution.filters}}, kernel_size={{model.layers[i].convolution.size}}{% if model.layers[i].convolution.activation%}, activation="{{model.layers[i].convolution.activation}}"{% endif %}{% if i == 0 %}{% block input_shape3d %}{% endblock %}{% endif %})){% endif %}
{% elif 'activation' in model.layers[i].keys() %}model.add(Activation("{{model.layers[i].activation}}"))
{% elif 'pooling' in model.layers[i].keys() %}{% if model.layers[i].pooling.size is integer and model.layers[i].pooling.type == 'max' %}model.add(MaxPooling1D({{model.layers[i].pooling.size}})){% elif model.layers[i].pooling.size is integer and model.layers[i].pooling.type == 'average' %}model.add(AveragePooling1D({{model.layers[i].pooling.size}})){% elif model.layers[i].pooling.size|length == 2 and model.layers[i].pooling.type == 'max' %}model.add(MaxPooling2D({{model.layers[i].pooling.size}})){% elif model.layers[i].pooling.size|length == 2 and model.layers[i].pooling.type == 'average' %}model.add(AveragePooling2D({{model.layers[i].pooling.size}})){% elif model.layers[i].pooling.size|length == 3 and model.layers[i].pooling.type == 'max' %}model.add(MaxPooling3D({{model.layers[i].pooling.size}})){% elif model.layers[i].pooling.size|length == 3 and model.layers[i].pooling.type == 'average' %}model.add(AveragePooling3D({{model.layers[i].pooling.size}})){% endif %}
{% elif 'batch_normalization' in model.layers[i].keys() %}model.add(BatchNormalization({{model.layers[i].batch_normalization.axis}})){% endif %}{%  endfor %}

model.compile(
            loss={% if model.loss == 'categorical_crossentropy' %}keras.losses.CategoricalCrossentropy(){% else %}keras.losses.MeanSquaredError(){% endif %},
            optimizer={% if model.optimizer.name == 'adam' %}keras.optimizers.Adam({% if model.optimizer.learning_rate %}learning_rate={{model.optimizer.learning_rate}}{% endif %}){% elif model.optimizer.name == 'adadelta' %}keras.optimizers.Adadelta({% if model.optimizer.learning_rate %}learning_rate={{model.optimizer.learning_rate}}{% endif %}){% elif model.optimizer.name == 'sgd' %}keras.optimizers.SGD({% if model.optimizer.learning_rate %}learning_rate={{model.optimizer.learning_rate}}{% endif %}){% elif model.optimizer.name == 'rmsprop' %}keras.optimizers.RMSprop({% if model.optimizer.learning_rate %}learning_rate={{model.optimizer.learning_rate}}{% endif %}){% endif %},
            metrics=['accuracy'])

model.summary()

## Training the model on the Training set

In [ ]:
{% block train %}
{% endblock %}

## Evaluating the Train Performance

In [ ]:
{% block evaluate_train %}
{% endblock %}

## Predicting the Test set results

In [ ]:
{% block predict %}
{% endblock %}

## Evaluating the Model Performance

In [ ]:
{% block evaluate %}
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix
import seaborn as sns

acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')**0.5
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'Accuracy Score: {acc}')
print(f'Precision Score: {precision}')
print(f'Recall Score: {recall}')
print(f'F1 Score: {f1}')

cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm)           
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');           
plt.show()

time_stop = process_time()
cpu_time = round(time_stop - time_start, 2)
print(f'Elapsed CPU Time: {cpu_time} seconds')
{% endblock %}

## Saving Model Statistics

In [ ]:
{% block save %}
import os
import json

path = 'statistics'
if not os.path.exists(path):
    os.mkdir(path)

stats = {
    "Accuracy Score": acc,
    "Precision Score": precision,
    "Recall Score": recall,
    "F1 Score": f1,
    "cpu time": cpu_time,
    "predicted": y_pred.flatten().tolist(), 
    "real": y_true.to_numpy().flatten().tolist()
}

with open(os.path.join(path, "cnn_da86a2e0-7ca6-11ec-b087-d7c57270a0f7.json"), "w") as f:
    json.dump(stats, f, ensure_ascii=False, indent=4)
{% endblock %}